# AI Config Targeting - Cookbook

This cookbook contains all code from `aiconfig-targeting/SKILL.md` for configuring targeting rules.

## Prerequisites
- `LAUNCHDARKLY_API_TOKEN`: API token with `ai-configs:write` permission
- AI Config with variations (run `cookbook_aiconfig_variations.ipynb` first)
- For segment targeting: segment created (run `cookbook_aiconfig_segments.ipynb` first)

In [14]:
# Install and load environment
%pip install requests python-dotenv -q

import os
from pathlib import Path
from dotenv import load_dotenv

def find_repo_root(start_path: Path = None) -> Path:
    current = start_path or Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / '.git').exists():
            return parent
    return current

repo_root = find_repo_root()
load_dotenv(repo_root / '.env')
print(f"[OK] Loaded environment from {repo_root / '.env'}")


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[OK] Loaded environment from /Users/ld_scarlett/Documents/Github/agent-skills/.env


---
## Setup and Configuration
From: `SKILL.md` lines 50-133

In [15]:
import requests
import json
import os
from typing import Dict, List, Optional

class AIConfigTargeting:
    """Manager for AI Config targeting rules"""

    def __init__(self, api_token: str, project_key: str):
        self.api_token = api_token
        self.project_key = project_key
        self.base_url = "https://app.launchdarkly.com/api/v2"
        self.headers = {
            "Authorization": api_token,
            "Content-Type": "application/json; domain-model=launchdarkly.semanticpatch"
        }

    def get_targeting(self, config_key: str) -> Optional[Dict]:
        """Get current targeting configuration including variation IDs"""
        url = f"{self.base_url}/projects/{self.project_key}/ai-configs/{config_key}/targeting"
        response = requests.get(url, headers={"Authorization": self.api_token})
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error getting targeting: {response.status_code} - {response.text}")
            return None

    def get_variation_id(self, config_key: str, variation_name: str) -> Optional[str]:
        """Look up variation ID from variation name or key"""
        targeting = self.get_targeting(config_key)
        if targeting and 'variations' in targeting:
            for var in targeting['variations']:
                var_key = var.get('key') or var.get('name') or var.get('_key')
                if var_key == variation_name:
                    return var.get('_id')
        return None

    def update_targeting(self, config_key: str, environment: str,
                        instructions: List[Dict], comment: str = "") -> Optional[Dict]:
        """Update targeting with semantic patch instructions"""
        url = f"{self.base_url}/projects/{self.project_key}/ai-configs/{config_key}/targeting"
        payload = {
            "environmentKey": environment,
            "instructions": instructions,
            "comment": comment
        }
        response = requests.patch(url, headers=self.headers, json=payload)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None

    def clear_all_rules(self, config_key: str, environment: str) -> bool:
        """Remove all targeting rules from an environment using replaceRules"""
        instructions = [{"kind": "replaceRules", "rules": []}]
        result = self.update_targeting(config_key, environment, instructions, "Clear all rules")
        if result:
            print(f"[OK] Cleared all rules")
            return True
        return False

    def clear_all_targets(self, config_key: str, environment: str) -> bool:
        """Remove all individual targets from an environment using replaceTargets"""
        instructions = [{"kind": "replaceTargets", "targets": []}]
        result = self.update_targeting(config_key, environment, instructions, "Clear all targets")
        if result:
            print(f"[OK] Cleared all targets")
            return True
        return False

# Initialize
targeting = AIConfigTargeting(
    api_token=os.environ["LAUNCHDARKLY_API_TOKEN"],
    project_key="support-ai"
)
print("[OK] AIConfigTargeting class initialized")

[OK] AIConfigTargeting class initialized


---
## View Current Targeting
From: `SKILL.md` lines 136-168

In [16]:
def view_targeting(self, config_key: str) -> Dict:
    """View current targeting configuration"""

    targeting_config = self.get_targeting(config_key)

    if targeting_config:
        print(f"[INFO] Targeting for '{config_key}':")

        variations = targeting_config.get('variations', [])

        # Show environments
        if 'environments' in targeting_config:
            for env_key, env_data in targeting_config['environments'].items():
                print(f"\n  Environment: {env_key}")
                print(f"    On: {env_data.get('on', False)}")
                print(f"    Rules: {len(env_data.get('rules', []))}")

                # Look up default variation name from index
                fallthrough_idx = env_data.get('fallthrough', {}).get('variation')
                if fallthrough_idx is not None and fallthrough_idx < len(variations):
                    var = variations[fallthrough_idx]
                    default_var = var.get('key') or var.get('name') or var.get('_key') or f'index {fallthrough_idx}'
                else:
                    default_var = 'Not set'
                print(f"    Default variation: {default_var}")

        return targeting_config
    return {}

# Bind to class
AIConfigTargeting.view_targeting = view_targeting

# Test with REAL config and clear existing rules first
print("=== Testing view_targeting with REAL config ===")
result = targeting.view_targeting("content-assistant")
if result:
    # Get environment key for cleanup
    env_key = list(result.get('environments', {}).keys())[0] if result.get('environments') else None
    if env_key:
        print(f"\n--- Clearing existing rules and targets in '{env_key}' ---")
        targeting.clear_all_rules("content-assistant", env_key)
        targeting.clear_all_targets("content-assistant", env_key)
    
    print(f"\n[OK] view_targeting() works")
    # Show variation info
    if 'variations' in result:
        print(f"\nVariations ({len(result['variations'])}):")
        for var in result['variations']:
            var_name = var.get('key') or var.get('name') or var.get('_key') or 'unknown'
            print(f"  {var_name}: {var.get('_id')}")
else:
    print(f"[WARNING] Could not get targeting config")

=== Testing view_targeting with REAL config ===
[INFO] Targeting for 'content-assistant':

  Environment: production
    On: False
    Rules: 4
    Default variation: Default Configuration

  Environment: test
    On: False
    Rules: 0
    Default variation: Default Configuration

--- Clearing existing rules and targets in 'production' ---
[OK] Cleared all rules
[OK] Cleared all targets

[OK] view_targeting() works

Variations (4):
  disabled: a109d7ce-de57-4831-bd71-5a074b2b26a0
  Default Configuration: 50adccf6-0312-4672-8d21-107361c99424
  Creative Style: dceffc34-3ba8-4a8b-b8c7-9ef8707cec2d
  Professional Style: 4895d9b9-5e0f-4b84-b82d-dbc8e12cd210


---
## Add Rule with Percentage Rollout
From: `SKILL.md` lines 173-233

In [17]:
def add_rollout_rule(self, config_key: str, environment: str,
                     clauses: List[Dict], rollout_weights: Dict[str, int],
                     rollout_context_kind: str = "user",
                     rollout_bucket_by: str = "key") -> Optional[Dict]:
    """
    Add a rule with percentage-based rollout

    Args:
        config_key: The AI Config key
        environment: Environment key
        clauses: List of clause conditions
        rollout_weights: Dict of variation_id: weight (in thousandths, 0-100000)
                        e.g., {"uuid-1": 60000, "uuid-2": 40000} for 60/40 split
        rollout_context_kind: Context kind for bucketing (default: "user")
        rollout_bucket_by: Attribute to bucket by (default: "key")
    """

    instructions = [
        {
            "kind": "addRule",
            "clauses": clauses,
            "rolloutWeights": rollout_weights,
            "rolloutContextKind": rollout_context_kind,
            "rolloutBucketBy": rollout_bucket_by
        }
    ]

    result = self.update_targeting(
        config_key,
        environment,
        instructions,
        comment="Add rule with percentage rollout"
    )

    if result:
        print(f"[OK] Rollout rule created")
    return result

# Bind to class
AIConfigTargeting.add_rollout_rule = add_rollout_rule
print("[OK] add_rollout_rule() defined")

[OK] add_rollout_rule() defined


In [18]:
# Test add_rollout_rule with REAL config
print("=== Testing add_rollout_rule ===")
targeting_data = targeting.get_targeting("content-assistant")
if targeting_data and 'variations' in targeting_data:
    variations = targeting_data.get('variations', [])
    if len(variations) >= 2:
        env_key = list(targeting_data['environments'].keys())[0]
        # Get variation IDs - handle different field names
        var_id_list = [v['_id'] for v in variations]
        
        print(f"Environment: {env_key}")
        print(f"Variation IDs: {var_id_list[:2]}")
        
        result = targeting.add_rollout_rule(
            config_key="content-assistant",
            environment=env_key,
            clauses=[{
                "contextKind": "user",
                "attribute": "tier",
                "op": "in",
                "values": ["cookbook-rollout-test"],
                "negate": False
            }],
            rollout_weights={
                var_id_list[0]: 60000,
                var_id_list[1]: 40000
            }
        )
        if result:
            print(f"[OK] add_rollout_rule() works")
        else:
            print("[INFO] Rollout rule may have failed")
    else:
        print("[INFO] Need at least 2 variations")
else:
    print("[WARNING] Could not get targeting data")

=== Testing add_rollout_rule ===
Environment: production
Variation IDs: ['a109d7ce-de57-4831-bd71-5a074b2b26a0', '50adccf6-0312-4672-8d21-107361c99424']
[OK] Rollout rule created
[OK] add_rollout_rule() works


---
## Target by User Attributes
From: `SKILL.md` lines 237-293

In [19]:
def add_attribute_rule(self, config_key: str, environment: str,
                       attribute: str, operator: str, values: List,
                       variation_id: str, context_kind: str = "user") -> Optional[Dict]:
    """
    Add a targeting rule based on context attributes

    Args:
        config_key: The AI Config key
        environment: Environment key
        attribute: Context attribute to match (e.g., "tier", "region")
        operator: Comparison operator ("in", "equals", "greaterThan", etc.)
        values: Values to match against
        variation_id: Variation ID (UUID) to serve when rule matches
        context_kind: Type of context ("user", "organization", etc.)
    """

    instructions = [
        {
            "kind": "addRule",
            "clauses": [
                {
                    "contextKind": context_kind,
                    "attribute": attribute,
                    "op": operator,
                    "values": values,
                    "negate": False
                }
            ],
            "variationId": variation_id
        }
    ]

    result = self.update_targeting(
        config_key,
        environment,
        instructions,
        comment=f"Add rule: {attribute} {operator} rule"
    )

    if result:
        print(f"[OK] Targeting rule created")
        print(f"  If {attribute} {operator} {values} -> variation {variation_id}")
    return result

# Bind to class
AIConfigTargeting.add_attribute_rule = add_attribute_rule
print("[OK] add_attribute_rule() defined")

[OK] add_attribute_rule() defined


In [20]:
# Test add_attribute_rule with REAL config
print("=== Testing add_attribute_rule ===")
targeting_data = targeting.get_targeting("content-assistant")
if targeting_data and 'variations' in targeting_data:
    variations = targeting_data.get('variations', [])
    if variations:
        env_key = list(targeting_data['environments'].keys())[0]
        var_id = variations[0].get('_id')
        
        result = targeting.add_attribute_rule(
            config_key="content-assistant",
            environment=env_key,
            attribute="tier",
            operator="in",
            values=["cookbook-premium-test"],
            variation_id=var_id
        )
        if result:
            print(f"[OK] add_attribute_rule() works")
        else:
            print("[INFO] Attribute rule may have failed")
    else:
        print("[INFO] No variations found")
else:
    print("[WARNING] Could not get targeting data")

=== Testing add_attribute_rule ===
[OK] Targeting rule created
  If tier in ['cookbook-premium-test'] -> variation a109d7ce-de57-4831-bd71-5a074b2b26a0
[OK] add_attribute_rule() works


---
## Target Individual Contexts
From: `SKILL.md` lines 297-340

In [21]:
def target_individuals(self, config_key: str, environment: str,
                      variation_id: str, context_keys: List[str],
                      context_kind: str = "user") -> Optional[Dict]:
    """
    Target specific context keys with a variation

    Args:
        config_key: The AI Config key
        environment: Environment key
        variation_id: Variation ID (UUID) to serve
        context_keys: List of context keys to target
        context_kind: Type of context ("user", "organization", etc.)
    """

    instructions = [
        {
            "kind": "addTargets",
            "variationId": variation_id,
            "contextKind": context_kind,
            "values": context_keys
        }
    ]

    result = self.update_targeting(
        config_key,
        environment,
        instructions,
        comment=f"Add individual targets for {len(context_keys)} contexts"
    )

    if result:
        print(f"[OK] Individual targeting configured for {len(context_keys)} contexts")
    return result

# Bind to class
AIConfigTargeting.target_individuals = target_individuals
print("[OK] target_individuals() defined")

[OK] target_individuals() defined


In [22]:
# Test target_individuals with REAL config
print("=== Testing target_individuals ===")
targeting_data = targeting.get_targeting("content-assistant")
if targeting_data and 'variations' in targeting_data:
    variations = targeting_data.get('variations', [])
    if variations:
        env_key = list(targeting_data['environments'].keys())[0]
        var_id = variations[0].get('_id')
        
        result = targeting.target_individuals(
            config_key="content-assistant",
            environment=env_key,
            variation_id=var_id,
            context_keys=["cookbook-test-user-1", "cookbook-test-user-2"]
        )
        if result:
            print(f"[OK] target_individuals() works")
        else:
            print("[INFO] Individual targeting may have failed")
    else:
        print("[INFO] No variations found")
else:
    print("[WARNING] Could not get targeting data")

=== Testing target_individuals ===
[OK] Individual targeting configured for 2 contexts
[OK] target_individuals() works


---
## Multi-Context Targeting
From: `SKILL.md` lines 344-400

In [23]:
def add_multi_context_rule(self, config_key: str, environment: str,
                          clauses: List[Dict],
                          variation_id: str) -> Optional[Dict]:
    """
    Add a rule targeting multiple context kinds

    Args:
        config_key: The AI Config key
        environment: Environment key
        clauses: List of clause definitions with context kinds
        variation_id: Variation ID (UUID) to serve when all clauses match
    """

    instructions = [
        {
            "kind": "addRule",
            "clauses": clauses,
            "variationId": variation_id
        }
    ]

    result = self.update_targeting(
        config_key,
        environment,
        instructions,
        comment="Add multi-context rule"
    )

    if result:
        print(f"[OK] Multi-context rule created")
    return result

# Bind to class
AIConfigTargeting.add_multi_context_rule = add_multi_context_rule
print("[OK] add_multi_context_rule() defined")

[OK] add_multi_context_rule() defined


In [24]:
# Test add_multi_context_rule with REAL config
print("=== Testing add_multi_context_rule ===")
targeting_data = targeting.get_targeting("content-assistant")
if targeting_data and 'variations' in targeting_data:
    variations = targeting_data.get('variations', [])
    if variations:
        env_key = list(targeting_data['environments'].keys())[0]
        var_id = variations[0].get('_id')
        
        result = targeting.add_multi_context_rule(
            config_key="content-assistant",
            environment=env_key,
            clauses=[
                {
                    "contextKind": "user",
                    "attribute": "role",
                    "op": "in",
                    "values": ["cookbook-admin"],
                    "negate": False
                },
                {
                    "contextKind": "user",
                    "attribute": "tier",
                    "op": "in",
                    "values": ["cookbook-enterprise"],
                    "negate": False
                }
            ],
            variation_id=var_id
        )
        if result:
            print(f"[OK] add_multi_context_rule() works")
        else:
            print("[INFO] Multi-context rule may have failed")
    else:
        print("[INFO] No variations found")
else:
    print("[WARNING] Could not get targeting data")

=== Testing add_multi_context_rule ===
[OK] Multi-context rule created
[OK] add_multi_context_rule() works


---
## Segment-Based Targeting
From: `SKILL.md` lines 405-455

In [25]:
def target_segments(self, config_key: str, environment: str,
                   segment_keys: List[str], variation_id: str,
                   include: bool = True) -> Optional[Dict]:
    """
    Target pre-defined segments

    Args:
        config_key: The AI Config key
        environment: Environment key
        segment_keys: List of segment keys to target (must be created first)
        variation_id: Variation ID (UUID) to serve to segment members
        include: True to include segments, False to exclude
    """

    instructions = [
        {
            "kind": "addRule",
            "clauses": [
                {
                    "op": "segmentMatch",
                    "values": segment_keys,
                    "negate": not include
                }
            ],
            "variationId": variation_id
        }
    ]

    result = self.update_targeting(
        config_key,
        environment,
        instructions,
        comment=f"{'Include' if include else 'Exclude'} segments: {', '.join(segment_keys)}"
    )

    if result:
        action = "included" if include else "excluded"
        print(f"[OK] Segments {action}: {segment_keys}")
    return result

# Bind to class
AIConfigTargeting.target_segments = target_segments
print("[OK] target_segments() defined")

[OK] target_segments() defined


In [26]:
# Test target_segments with REAL config
# Note: Uses segment created in cookbook_aiconfig_segments.ipynb
print("=== Testing target_segments ===")
targeting_data = targeting.get_targeting("content-assistant")
if targeting_data and 'variations' in targeting_data:
    variations = targeting_data.get('variations', [])
    if variations:
        env_key = list(targeting_data['environments'].keys())[0]
        var_id = variations[0].get('_id')
        
        result = targeting.target_segments(
            config_key="content-assistant",
            environment=env_key,
            segment_keys=["cookbook-beta-testers"],
            variation_id=var_id
        )
        if result:
            print(f"[OK] target_segments() works")
        else:
            print("[INFO] Segment targeting may have failed (run cookbook_aiconfig_segments.ipynb first)")
    else:
        print("[INFO] No variations found")
else:
    print("[WARNING] Could not get targeting data")

=== Testing target_segments ===
[OK] Segments included: ['cookbook-beta-testers']
[OK] target_segments() works
